In [1]:
from datetime import datetime, timedelta

In [2]:
import calendar

In [3]:
import requests

In [4]:
import grequests

In [5]:
import pandas as pd

In [6]:
from pandas import DataFrame, Series

In [7]:
import json

In [8]:
validParameters = ["starttime", "endtime", "updatetime", "minlatitude", "maxlatitude", "minlongitude", "maxlongitude"]

In [9]:
parameters = {
    "starttime2": "2015-1-1", 
    "endtime2": "2015-12-1", 
    "products": "focal-mechanism",
    "minlongitude": "-125.69510773828125",
    "maxlongitude": "-119.31754426171875",
    "minlatitude": "36.682994957749884",
    "maxlatitude": "38.79394764206436",
}

In [10]:
MAX_EVENTS = 10000
TIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATE_FORMAT = '%Y-%m-%d'

In [11]:
def validDate(dt):
    if type(dt) is datetime:
        return true
    
    if type(dt) is str:
        try:
            parsed = datetime.strptime(dt,TIME_FORMAT)
        except:
            try:
                parsed = datetime.strptime(dt,DATE_FORMAT)
            except:
                return False
        return True

    if type(dt) is int:
        try:
            parsed = datetime.fromtimestamp(dt)
        except:
            return False

In [12]:
def prepareParameters(parameters):
    if parameters is None:
        parameters = {}
    elif type(parameters) is not dict:
        raise Exception("Expecting a dictionary of parameter values")
       
    if ("starttime" not in parameters) or (not validDate(parameters["starttime"])):
        parameters["starttime"] = datetime(2013, 1, 1).strftime(DATE_FORMAT)
    if ("endtime" not in parameters) or (not validDate(parameters["endtime"])):
        parameters["endtime"] = datetime.utcnow().strftime(DATE_FORMAT)
        
    return parameters

In [13]:
def splitDateRange(start, end, interval):
    if (not validDate(start)) and (not validDate(end)):
        raise Exception("Invalid date range: {0}-{1}".format(start, end))
    
    startdatetime = datetime.strptime(start, DATE_FORMAT)
    enddatetime = datetime.strptime(end, DATE_FORMAT)
    num_days = (enddatetime - startdatetime).days
    
    date_list = [(startdatetime + timedelta(days=x*interval), startdatetime + timedelta(days=(x+1)*interval)) 
                 for x in range(0, int(num_days / interval))]
    
    #append final date interval for remainder of days
    if date_list[-1][1] < enddatetime:
        date_list.append((date_list[-1][1], enddatetime))
        
    return date_list

In [14]:
def createQueries(params):
    # check the number of events returned by the query.  if over the max limit, split request into time segments
    # and return the list of requests as an array
    # if the request is under the max limit, return an array of 1, which will contain the original query
    queries = []
    
    params = prepareParameters(params)
    
    queryString = "&".join(["{0}={1}".format(key, params[key]) for key in params if key in validParameters])
    
    countUrl = "http://earthquake.usgs.gov/fdsnws/event/1/count?%s" % queryString
    print(countUrl)
    result = requests.get(countUrl)
    
    count = 0
    if result.status_code == 200:
        count = int(result.text)
    else:
        print("Error getting count: [{0}] {1}".format(result.status_code, result.text))
    
    queryUrl = "http://earthquake.usgs.gov/fdsnws/event/1/query.geojson?"
    
    if count > MAX_EVENTS:
        date_list = splitDateRange(params["starttime"], params["endtime"], 30)
        print("Need to break up queries. {0} results found.".format(count))
        for startdate, enddate in date_list:
            cpparams = params.copy()
            cpparams["starttime"] = startdate.strftime(DATE_FORMAT)
            cpparams["endtime"] = enddate.strftime(DATE_FORMAT)
            queryString = "&".join(["{0}={1}".format(key, cpparams[key]) for key in cpparams if key in validParameters])
            
            queries.append("{0}{1}".format(queryUrl, queryString))
    elif count == 0:
        print("No results match that query.")
    else:
        print("Single query is fine. {0} results found.".format(count))
        queries.append("{0}{1}".format(queryUrl, queryString))
        
    return queries

In [15]:
def processSummaryDataResponse(response, **data):
    if response.status_code != 200:
        print("ERROR processing detail.  Status: {0} Text: {1}".format(response.status_code, response.text))
        return

    return response

In [16]:
def requestErrorHandler(response, exception):
    print("Request failed ({0}): {1}".format(url, exception))

In [17]:
def loadEventData(params):
    parameters = prepareParameters(params)
    summaryUrlList = createQueries(parameters)
    
    masterDataFrame = None
    
    #use first request to initialize DataFrame and then queue up remaining queries
    response = requests.get(summaryUrlList[0])
    if response.status_code == 200:
        json_data = response.json()

        features = DataFrame(json_data["features"])
        main = pd.concat([features, features["properties"].apply(pd.Series)], axis=1)

        masterDataFrame = DataFrame(main)

        # process the remaining requests by sending them asynchronously through grequests
        summaryUrlRequests = [grequests.get(url, hooks = {'response': processSummaryDataResponse }) 
                              for url in summaryUrlList[1:]]

        for resp in grequests.imap(summaryUrlRequests, exception_handler=requestErrorHandler):
            json_data = resp.json()

            features = DataFrame(json_data["features"])
            main = pd.concat([features, features["properties"].apply(pd.Series)], axis=1)

            masterDataFrame = masterDataFrame.append(main, ignore_index = True)
            print(masterDataFrame.shape)
            
    return masterDataFrame

In [18]:
eventData = loadEventData(parameters)

http://earthquake.usgs.gov/fdsnws/event/1/count?maxlongitude=-119.31754426171875&maxlatitude=38.79394764206436&starttime=2013-01-01&minlatitude=36.682994957749884&minlongitude=-125.69510773828125&endtime=2016-03-31
Need to break up queries. 17137 results found.
(390, 30)
(888, 30)
(1324, 30)
(1767, 30)
(2187, 30)
(2612, 30)
(3067, 30)
(3552, 30)
(4027, 30)
(4522, 30)
(4941, 30)
(5397, 30)
(5858, 30)
(6305, 30)
(6682, 30)
(7055, 30)
(7514, 30)
(7972, 30)
(8449, 30)
(9197, 30)
(9583, 30)
(10034, 30)
(10497, 30)
(10912, 30)
(11307, 30)
(11778, 30)
(12212, 30)
(12638, 30)
(13143, 30)
(13522, 30)
(13954, 30)
(14353, 30)
(14747, 30)
(15127, 30)
(15879, 30)
(16270, 30)
(16564, 30)
(16953, 30)
(17137, 30)


In [19]:
eventData

,geometry,id,properties,type,alert,cdi,code,detail,dmin,felt,...,sources,status,time,title,tsunami,type,types,tz,updated,url
0,"{'type': 'Point', 'coordinates': [-121.5548, 3...",nc71929426,"{'nst': None, 'ids': ',nc71929426,usp000jyuv,c...",Feature,None,3.1,71929426,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.035933,10,...,",nc,us,ci,",REVIEWED,1359251751100,"M 2.8 - 2km SSW of San Juan Bautista, California",0,earthquake,",dyfi,general-link,geoserve,impact-text,nearby...",-480,1457699561770,http://earthquake.usgs.gov/earthquakes/eventpa...
1,"{'type': 'Point', 'coordinates': [-122.36, 37....",usp000jyuq,"{'nst': 18, 'ids': ',nc71929271,usp000jyuq,', ...",Feature,None,2.7,p000jyuq,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,13,...,",nc,us,",reviewed,1359225564290,M 2.6 - Northern California,0,earthquake,",dyfi,impact-text,origin,phase-data,",NaN,1422580916689,http://earthquake.usgs.gov/earthquakes/eventpa...
2,"{'type': 'Point', 'coordinates': [-122.2333, 3...",nc71928861,"{'nst': None, 'ids': ',nc71928861,', 'magType'...",Feature,None,3.3,71928861,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.008983,4,...,",nc,",REVIEWED,1359161957300,"M 1.8 - 3km ENE of Berkeley, California",0,earthquake,",dyfi,general-link,general-link,geoserve,nearb...",-480,1422653773801,http://earthquake.usgs.gov/earthquakes/eventpa...
3,"{'type': 'Point', 'coordinates': [-122.059, 38...",usp000jyts,"{'nst': 27, 'ids': ',nc71928651,usp000jyts,', ...",Feature,None,3.4,p000jyts,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,369,...,",nc,us,",reviewed,1359135538700,M 2.9 - Northern California,0,earthquake,",cap,dyfi,impact-text,origin,phase-data,",NaN,1422439119296,http://earthquake.usgs.gov/earthquakes/eventpa...
4,"{'type': 'Point', 'coordinates': [-122.395, 38...",usp000jytk,"{'nst': 5, 'ids': ',nc71928466,usp000jytk,', '...",Feature,None,2.4,p000jytk,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,3,...,",nc,us,",reviewed,1359106538230,M 2.0 - Northern California,0,earthquake,",dyfi,impact-text,origin,phase-data,",NaN,1422670638787,http://earthquake.usgs.gov/earthquakes/eventpa...
5,"{'type': 'Point', 'coordinates': [-119.6145, 3...",nc71069629,"{'nst': None, 'ids': ',nc71069629,', 'magType'...",Feature,None,NaN,71069629,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nc,",REVIEWED,1359093554800,"M 0.0 - 33km SW of Smith Valley, Nevada",0,earthquake,",general-link,general-link,general-link,geoser...",-480,1360380540590,http://earthquake.usgs.gov/earthquakes/eventpa...
6,"{'type': 'Point', 'coordinates': [-119.6177, 3...",nc71928386,"{'nst': None, 'ids': ',nn00400861,nc71928386,'...",Feature,None,NaN,71928386,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nn,nc,",REVIEWED,1359093513900,"M 1.5 - 35km SW of Smith Valley, Nevada",0,earthquake,",cap,general-link,general-link,general-link,ge...",-480,1400530246000,http://earthquake.usgs.gov/earthquakes/eventpa...
7,"{'type': 'Point', 'coordinates': [-122.232, 37...",usp000jytd,"{'nst': 12, 'ids': ',nc71928376,usp000jytd,', ...",Feature,None,2.0,p000jytd,http://earthquake.usgs.gov/fdsnws/event/1/quer...,NaN,7,...,",nc,us,",reviewed,1359090747530,"M 2.0 - San Francisco Bay area, California",0,earthquake,",cap,dyfi,impact-text,origin,phase-data,",NaN,1422611894841,http://earthquake.usgs.gov/earthquakes/eventpa...
8,"{'type': 'Point', 'coordinates': [-119.6242, 3...",nc71928341,"{'nst': None, 'ids': ',nc71928341,', 'magType'...",Feature,None,NaN,71928341,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nc,",REVIEWED,1359087199600,"M 1.9 - 35km SW of Smith Valley, Nevada",0,earthquake,",general-link,general-link,general-link,geoser...",-480,1360380189213,http://earthquake.usgs.gov/earthquakes/eventpa...
9,"{'type': 'Point', 'coordinates': [-119.618, 38...",nc71928221,"{'nst': None, 'ids': ',nc71928221,nn00400826,'...",Feature,None,NaN,71928221,http://earthquake.usgs.gov/fdsnws/event/1/quer...,0.152714,NaN,...,",nc,n